In [103]:
%matplotlib inline
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn import model_selection, ensemble
from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb



In [78]:
xytte = pd.read_csv("data/xytte.csv")

####################### ВНИМАНИЕ #######################
# здесь трейн это которые в задаче трейн, с ответами
# тест без ответов
# но дальше в алгоритмах трейн будет разбиваться на два подсета
# которые будут у меня называться x1-y1 и x2-y2, а по смыслу как раз трейн и тест
# надо не перепутать

x_train = xytte[xytte.returned == xytte.returned].reset_index().drop("returned", axis=1)
y_train = xytte[xytte.returned == xytte.returned].reset_index()[["returned"]]
x_test  = xytte[xytte.returned != xytte.returned].reset_index().drop("returned", axis=1)

In [79]:
# проверяем, что нигде не налажали
# читаем из файла оригинальные данные

x_test_ref = pd.read_csv("data/x_test.csv", sep=";", dtype=np.float32)
x_train_ref = pd.read_csv("data/x_train.csv", sep=";", dtype=np.float32)
y_train_ref = pd.read_csv("data/y_train.csv", sep=";", dtype=np.float32, header=None, names=["returned"])

# заводим функцию на сравнение таблиц

def df_equal(df1, df2):
    if list(df1.columns) != list(df2.columns): 
        print("Mismatch columns:")
        print("df1: ", list(df1.columns))
        print("df2: ", list(df2.columns))
        return False
    if list(df1.index) != list(df2.index): 
        print("Mismatch index:")
        print("df1: ", list(df1.columns)[0], ":", list(df1.columns)[-1])
        print("df2: ", list(df2.columns)[0], ":", list(df2.columns)[-1])
        return False
    
    ne_stacked = (df1 != df2).stack()
    changed = ne_stacked[ne_stacked]
    changed.index.names = ['id', 'col']

    difference_locations = np.where(df1 != df2)
    changed_from = df1.values[difference_locations]
    changed_to = df2.values[difference_locations]
    diff = pd.DataFrame({'from': changed_from, 'to': changed_to}, index=changed.index)
    return not diff[diff["from"] - diff["to"] > 0.000001].count()[0]

# ассертим наши таблицы

assert(df_equal(x_test[x_test_ref.columns], x_test_ref))
assert(df_equal(x_train[x_train_ref.columns], x_train_ref))
assert(df_equal(y_train, y_train_ref))

In [82]:
params_orig = [
     'maxPlayerLevel',
     'numberOfAttemptedLevels',
     'attemptsOnTheHighestLevel',
     'totalNumOfAttempts',
     'averageNumOfTurnsPerCompletedLevel',
     'numberOfBoostersUsed',
     'fractionOfUsefullBoosters',
     'totalScore',
     'totalBonusScore',
     'totalStarsCount',
     'numberOfDaysActuallyPlayed',
]
     
params_mod = [
     #'attemptsOnTheHighestLevel_dvd',
     'attemptsOnTheHighestLevel_ln',
     'attemptsPerDay',
     'averageNumOfTurnsPerCompletedLevel_dvd',
     'maxPlayerLevel_ln',
     'numberOfAttemptedLevels_dvd',
     #'numberOfBoostersUsed_dvd',
     'numberOfBoostersUsed_dvd_ln',
     #'numberOfBoostersUsed_ln',
     'numberOfDaysActuallyPlayed_ln',
     'totalBonusScore_dvd',
     'totalNumOfAttempts_ln',
     #'totalScore_ln',
     'totalScore_ln_dvd',
     #'totalStarsCount_dvd',
     'totalStarsCount_dvd_ln',
     #'totalStarsCount_ln',
]

params_bool = [
     'allAttemptsOnTheHighestLevel',
     'attLevelsMoreThanMaxLevel',
     'doReturnOnLowerLevels',
     'zeroTotalScore',
     'zeroTurnsPerCompletedLevel'
]

params_mod_norm = [
     'attemptsOnTheHighestLevel_dvd_norm',
     #'attemptsOnTheHighestLevel_ln_norm',
     #'attemptsOnTheHighestLevel_norm',
     'attemptsPerDay_norm',
     'averageNumOfTurnsPerCompletedLevel_dvd_norm',
     #'averageNumOfTurnsPerCompletedLevel_norm',
     'fractionOfUsefullBoosters_norm',
     'maxPlayerLevel_ln_norm',
     #'maxPlayerLevel_norm',
     'numberOfAttemptedLevels_dvd_norm',
     #'numberOfAttemptedLevels_norm',
     'numberOfBoostersUsed_dvd_ln_norm',
     #'numberOfBoostersUsed_dvd_norm',
     #'numberOfBoostersUsed_ln_norm',
     #'numberOfBoostersUsed_norm',
     'numberOfDaysActuallyPlayed_ln_norm',
     #'numberOfDaysActuallyPlayed_norm',
     'totalBonusScore_dvd_norm',
     #'totalBonusScore_norm',
     'totalNumOfAttempts_ln_norm',
     #'totalNumOfAttempts_norm',
     'totalScore_ln_dvd_norm',
     #'totalScore_ln_norm',
     #'totalScore_norm',
     'totalStarsCount_dvd_ln_norm',
     #'totalStarsCount_dvd_norm',
     'totalStarsCount_ln_norm',
     #'totalStarsCount_norm',
]

params = params_orig + ["doReturnOnLowerLevels"]

In [98]:
x_train_feed = x_train[params]
x_test_feed = x_test[params]
y_train_feed = list(y_train.returned)
#y_test_feed = list(y_test)

x1, x2, y1, y2 = train_test_split(x_train_feed, y_train_feed, test_size = 0.3)

N_train, _ = x1.shape 
N_test,  _ = x2.shape 
print(N_train, N_test)

17702 7587


In [100]:
# RANDOM FOREST

estimators_list = [20, 50, 100]
features_list = [3, 5, 7]
for estimators in estimators_list:
    for features in features_list:
        rf = ensemble.RandomForestClassifier(n_estimators=estimators, random_state=35, max_features=features)
        rf.fit(x1, y1)
        err_train = np.mean(y1 != rf.predict(x1))
        err_test  = np.mean(y2  != rf.predict(x2))
        print(err_train, err_test)

0.00875607276014 0.175695268222
0.00920799909615 0.178858573876
0.00847361880014 0.181494661922
0.00451926336007 0.173454593383
0.00463224494407 0.176881507842
0.00474522652808 0.17674970344
0.00423680940007 0.170291287729
0.00434979098407 0.175431659417
0.00429330019207 0.174640833004


In [101]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

print("Feature importances:")
for f, idx in enumerate(indices):
    print("{:2d}. feature '{:5s}' ({:.4f})".format(f + 1, params[idx], importances[idx]))

Feature importances:
 1. feature 'numberOfDaysActuallyPlayed' (0.2571)
 2. feature 'totalNumOfAttempts' (0.1626)
 3. feature 'averageNumOfTurnsPerCompletedLevel' (0.1100)
 4. feature 'totalScore' (0.1095)
 5. feature 'maxPlayerLevel' (0.0829)
 6. feature 'totalBonusScore' (0.0660)
 7. feature 'numberOfBoostersUsed' (0.0469)
 8. feature 'totalStarsCount' (0.0467)
 9. feature 'fractionOfUsefullBoosters' (0.0436)
10. feature 'attemptsOnTheHighestLevel' (0.0425)
11. feature 'numberOfAttemptedLevels' (0.0239)
12. feature 'doReturnOnLowerLevels' (0.0081)


In [104]:
# GRAD BOOST

gbt = ensemble.GradientBoostingClassifier(n_estimators=100, random_state=11)
gbt.fit(x1, y1)

err_train = np.mean(y1 != gbt.predict(x1))
err_test = np.mean(y2 != gbt.predict(x2))
print(err_train, err_test)

0.168173087787 0.164623698432


In [107]:
# XGBOOST

model = xgb.XGBClassifier()
model.fit(x1, y1)

y_pred = model.predict(x2)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y2, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

#log_loss(y_test_1D, predictions, eps=1e-15)

Accuracy: 83.64%
